In [ ]:
# Get a value from ENV
import os
import sys
from dotenv import load_dotenv
import google.auth
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from typing import List
import psycopg2
import django
import json

load_dotenv()

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

creds, _ = google.auth.default(scopes=SCOPES)

# Setup Django App
sys.path.append('../assistant')
os.environ["DJANGO_SETTINGS_MODULE"] = "config.settings"

django.setup()

# os.environ["DJANGO_SETTINGS_MODULE"]



In [38]:
class Customer:
    '''
    Attributes:
        customer_id (int): Unique identifier for the customer.
        first_name (str): The first name of the customer.
        last_name (str): The last name of the customer.
        preferred_name (str): The preferred name of the customer.
        status (bool): The status of the customer (e.g., active, inactive).
    '''
    def __init__(self, customer_id, first_name, last_name, preferred_name, status):
        self.customer_id = customer_id
        self.first_name = first_name
        self.last_name = last_name
        self.preferred_name = preferred_name
        self.status = status

    @property
    def name(self):
        return self.preferred_name if self.preferred_name else self.first_name
    
    def __str__(self):
        return f"{self.customer_id}# {self.name} {self.last_name}"

In [80]:
def get_spreadsheet() -> List:
    try:
        service = build("drive", "v3", credentials=creds)

        return (
            service.files()
            .list(
                fields="nextPageToken, files(id, name)",
                q="name contains '_tennis' and mimeType='application/vnd.google-apps.spreadsheet' and name != 'Template_Tennis'"
            )
            .execute()
            .get("files", [])
        )

    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f"An error occurred: {error}")

def get_sheet_data(spreadsheetId):
    try:
        service = build("sheets", "v4", credentials=creds)

        return (
            service.spreadsheets()
            .values()
            .get(spreadsheetId=spreadsheetId, range="Events")
            .execute()
            .get("values", [])
        )

    except HttpError as error:
        # TODO(developer) - Handle errors from sheets API.
        print(f"An error occurred: {error}")

def get_postgres_client():
    conn = psycopg2.connect(
        dbname=os.environ["PG_DB"],
        user=os.environ["PG_USER"],
        password=os.environ["PG_PASS"],
        host=os.environ["PG_HOST"],
        port=os.environ["PG_PORT"]
    )
    return conn

def get_customers() -> List[Customer]:
    customers = []

    with get_postgres_client() as conn:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM customers_customer")
            rows = cur.fetchall()

            for row in rows:
                status = True if row[-1] == "active" else False
                customer = Customer(*row[:-1], status)
                customers.append(customer)

    return customers

def get_all_sheets_data(spreadsheets) -> dict:
    all_data = {}
    for sheet in spreadsheets:
        sheet_id = sheet['id']
        sheet_data = get_sheet_data(sheet_id)
        all_data[sheet['name']] = sheet_data

    return all_data

def load_transactions() -> List:
    transactions = []
    with open('transactions.json', 'r') as f:
        transactions = json.load(f)

    return transactions

In [ ]:
spreadsheets = get_spreadsheet()
customers = get_customers()
all_sheets_data = get_all_sheets_data(spreadsheets)

TimeoutError: The read operation timed out